# Flash Point: Fire Rescue

## Instalación e importación de librerías

In [1]:
# Descargar e instalar mesa, seaborn y plotly

%pip install mesa==2.3.1 seaborn plotly --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Importamos las clases que se requieren para manejar los agentes (Agent) y su entorno (Model).
# Cada modelo puede contener múltiples agentes.
from mesa import Agent, Model

# Debido a que necesitamos que existe un solo agente por celda, elegimos ''SingleGrid''.
from mesa.space import SingleGrid

# Con 'SimultaneousActivation' podemos activar todos los agentes al mismo tiempo.
from mesa.time import SimultaneousActivation

# Con 'RandomActivation' podemos activar los agentes en un orden aleatorio.
from mesa.time import RandomActivation

# Haremos uso de ''DataCollector'' para obtener información de cada paso de la simulación.
from mesa.datacollection import DataCollector

# BATCH_RUNNER
from mesa.batchrunner import batch_run

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

# Importamos el paquete seaborn para mejorar la visualización de los datos.
import seaborn as sns

# Importamos el paquete plotly para crear gráficos interactivos.
import plotly.graph_objects as go

# Importamos el paquete networkx para crear grafos.
import networkx as nx

## Leer archivo de configuración inicial

In [3]:
# Abrir el archivo de configuración
file = open("board-config.txt", "r")

# Obtener la información del tablero
config = file.readlines()

# Cerrar el archivo
file.close()

# Crear un diccionario para almacenar la configuración del tablero
board_config = {}

# Configuración del tablero
board_config['board'] = [x.replace("\n", "").split() for x in config[:6]]

# Puntos de interés
board_config['points_of_interest'] = [x.replace("\n", "").split() for x in config[6:9]]

# Indicadores de fuego
board_config['fire_indicators'] = [x.replace("\n", "").split() for x in config[9:19]]

# Puertas
board_config['doors'] = [x.replace("\n", "").split() for x in config[19:27]]

# Puntos de entrada
board_config['entry_points'] = [x.replace("\n", "").split() for x in config[27:31]]

# Imprimir la configuración del tablero
board_config

{'board': [['1100', '1000', '1001', '1100', '1001', '1100', '1000', '1001'],
  ['0100', '0000', '0011', '0100', '0011', '0110', '0010', '0011'],
  ['0100', '0001', '1100', '1000', '1000', '1001', '1100', '1001'],
  ['0100', '0011', '0110', '0010', '0010', '0011', '0110', '0011'],
  ['1100', '1000', '1000', '1000', '1001', '1100', '1001', '1101'],
  ['0110', '0010', '0010', '0010', '0011', '0110', '0011', '0111']],
 'points_of_interest': [['2', '4', 'v'], ['5', '1', 'f'], ['5', '8', 'v']],
 'fire_indicators': [['2', '2'],
  ['2', '3'],
  ['3', '2'],
  ['3', '3'],
  ['3', '4'],
  ['3', '5'],
  ['4', '4'],
  ['5', '6'],
  ['5', '7'],
  ['6', '6']],
 'doors': [['1', '3', '1', '4'],
  ['2', '5', '2', '6'],
  ['2', '8', '3', '8'],
  ['3', '2', '3', '3'],
  ['4', '4', '5', '4'],
  ['4', '6', '4', '7'],
  ['6', '5', '6', '6'],
  ['6', '7', '6', '8']],
 'entry_points': [['1', '6'], ['3', '1'], ['4', '8'], ['6', '3']]}

In [4]:
doors = {}

for door in board_config['doors']:
    # Se resta 1 a las coordenadas para que coincidan con las coordenadas del tablero
    doors[(int(door[0]) - 1, int(door[1]) - 1)] = (int(door[2]) - 1, int(door[3]) - 1)


doors

{(0, 2): (0, 3),
 (1, 4): (1, 5),
 (1, 7): (2, 7),
 (2, 1): (2, 2),
 (3, 3): (4, 3),
 (3, 5): (3, 6),
 (5, 4): (5, 5),
 (5, 6): (5, 7)}

## Solución aleatoria

In [5]:
def initialize_board(board_config):
    """
    De acuerdo a la configuración del tablero, se inicializa el tablero como un grafo
    """

    # Crear un grafo vacío
    G = nx.Graph()

    # Información del tablero (transiciones y muros)
    # "TLDR" -> Top, Left, Down, Right
    board_info = board_config['board']

    # Puertas
    doors = {}
    for door in board_config['doors']:
        # Se resta 1 a las coordenadas para que coincidan con las coordenadas del tablero
        doors[(int(door[0]) - 1, int(door[1]) - 1)] = (int(door[2]) - 1, int(door[3]) - 1)

    # Agregar los nodos al grafo
    for i in range(len(board_info)):
        for j in range(len(board_info[i])):
            G.add_node((i, j))

    # Agregar las aristas al grafo
    for i in range(len(board_info)):
        for j in range(len(board_info[i])):

            # String con la información de la celda
            # "TLDR" -> Top, Left, Down, Right
            # Peso 1 -> Libre / Puerta abierta o destruida
            # Peso 2 -> Puerta cerrada
            # Peso 5 -> Pared
            
            # Verificar si hay una transición arriba
            if board_info[i][j][0] == '0':
                G.add_edge((i, j), (i - 1, j), weight = 1)
            else:
                # Verificar si hay una puerta
                if (i, j) in doors and doors[(i, j)] == (i - 1, j):
                    # Agregar puerta
                    G.add_edge((i, j), (i - 1, j), weight = 2)
                # Si no hay puerta, agregar pared
                elif i != 0 and not G.has_edge((i, j), (i - 1, j)):
                    G.add_edge((i, j), (i - 1, j), weight = 5)


            # Verificar si hay una transición a la izquierda
            if board_info[i][j][1] == '0':
                G.add_edge((i, j), (i, j - 1), weight = 1)
            else:
                # Verificar si hay una puerta
                if (i, j) in doors and doors[(i, j)] == (i, j - 1):
                    # Agregar puerta
                    G.add_edge((i, j), (i, j - 1), weight = 2)
                # Si no hay puerta, agregar pared
                elif j != 0 and not G.has_edge((i, j), (i, j - 1)):
                    G.add_edge((i, j), (i, j - 1), weight = 5)

            # Verificar si hay una transición abajo
            if board_info[i][j][2] == '0':
                G.add_edge((i, j), (i + 1, j), weight = 1)
            else:
                # Verificar si hay una puerta
                if (i, j) in doors and doors[(i, j)] == (i + 1, j):
                    # Agregar puerta
                    G.add_edge((i, j), (i + 1, j), weight = 2)
                # Si no hay puerta, agregar pared
                elif i != len(board_info) - 1 and not G.has_edge((i, j), (i + 1, j)):
                    G.add_edge((i, j), (i + 1, j), weight = 5)

            # Verificar si hay una transición a la derecha
            if board_info[i][j][3] == '0':
                G.add_edge((i, j), (i, j + 1), weight = 1)
            else:
                # Verificar si hay una puerta
                if (i, j) in doors and doors[(i, j)] == (i, j + 1):
                    # Agregar puerta
                    G.add_edge((i, j), (i, j + 1), weight = 2)
                # Si no hay puerta, agregar pared
                elif j != len(board_info[i]) - 1 and not G.has_edge((i, j), (i, j + 1)):
                    G.add_edge((i, j), (i, j + 1), weight = 5)

    # Retornar el grafo
    return G

In [6]:
# Inicializar el grafo
G = initialize_board(board_config)

# Definir las posiciones de los nodos como una cuadrícula
pos = {node: (node[1],-node[0]) for node in G.nodes()}  # El eje y se invierte para que la visualización sea de arriba a abajo

# Crear trazas para las aristas
edge_x = []
edge_y = []
edge_weights = []
for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)  # Para separar las líneas de las aristas
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)
    
    # Guardar la posición media para los pesos
    edge_weights.append(((x0 + x1) / 2, (y0 + y1) / 2, edge[2]['weight']))

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=1, color='#888'),
    hoverinfo='none',
    mode='lines'
)

# Crear trazas para los nodos
node_x = []
node_y = []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        size=10,
        colorbar=dict(
            thickness=15,
            title='Conexiones',
            xanchor='left',
            titleside='right'
        )
    )
)

# Agregar etiquetas a los nodos
node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append(f'Nodo {node}: {len(adjacencies[1])} conexiones')

node_trace.marker.color = node_adjacencies
node_trace.text = node_text

# Crear anotaciones para los pesos de las aristas
weight_annotations = []
for (x, y, weight) in edge_weights:
    weight_annotations.append(
        dict(
            x=x,
            y=y,
            text=str(weight),
            showarrow=False,
            font=dict(
                size=10,
                color='black'
            )
        )
    )

# Crear la figura
fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title='Grafo de la configuración del tablero en forma de cuadrícula con pesos',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    annotations=weight_annotations,
                    xaxis=dict(showgrid=False, zeroline=False),
                    yaxis=dict(showgrid=False, zeroline=False)
                ))

fig.show()


In [7]:
class BoardModel(Model):

    def __init__(self, board_config):

        # Definir el tamaño del tablero
        self.height = len(board_config['board'])
        self.width = len(board_config['board'][0])

        # Crear el grid
        self.grid = SingleGrid(self.width, self.height, torus=False)

        # Crear grafo para el tablero
        self.graph = nx.Graph()
        